In [8]:
import pandas as pd
import geopandas as gpd
import numpy as np
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import *
import psycopg2
%matplotlib inline

pd.options.display.max_columns = None

In [9]:
password = open('password.txt','r').readline()
username = 'kmerrem'

#conn = psycopg2.connect(host="localhost",database="gisdata", user="kmerrem", password=password)
#curs = conn.cursor()

#psycopg2 is used for postgres, can't use this conn to write with
# conn = psycopg2.connect(host="172.19.118.205",port="5432",database="gisdata", user="kmerrem", password=password)
# curs = conn.cursor()

#this is to write from geopandas to postgis table 
#gisdata_engine = create_engine('postgresql://%s:%s@172.19.118.205:5432/gisdata' %(username,password))
walkability_engine = create_engine('postgresql://%s:%s@172.19.118.205:5432/walkability' %(username,password))

In [10]:
buildings = gpd.read_postgis("select * from public.massing_dis_simp_1", walkability_engine, 'wkb_geometry')
cts = gpd.read_postgis("SELECT * FROM public.census_tracts_blank", engine, 'geom')


In [11]:
buildings.dtypes

ogc_fid           int64
objectid        float64
shape_leng      float64
shape_area      float64
wkb_geometry     object
dtype: object

In [12]:
censustracts = gpd.read_postgis("select * from public.census_tracts_blank", walkability_engine, 'geom')

In [15]:
buildings_cleaned = buildings[buildings.geometry.is_valid]

In [14]:
#http://geopandas.org/reference/geopandas.sjoin.html
census_buildings_cleaned = gpd.sjoin(buildings_cleaned, censustracts) 


In [16]:
census_buildings_cleaned

,ogc_fid,objectid,shape_leng_left,shape_area_left,wkb_geometry,index_right,gid,__gid,ctuid,ctname,pruid,prname,cmauid,cmapuid,cmaname,cmatype,shape_leng_right,shape_area_right,ctnum,geotext
0,243795,243774.0,271.07940,1728.41535,"(POLYGON ((327428.3540000003 4843863.946, 3274...",24,25,14,5350332.00,0332.00,35,Ontario,535,35535,Toronto,B,5720.576958,1.525748e+06,535033200.0,535033200
27,243822,243801.0,58.25455,176.75016,(POLYGON ((327217.7879999997 4843868.336999999...,24,25,14,5350332.00,0332.00,35,Ontario,535,35535,Toronto,B,5720.576958,1.525748e+06,535033200.0,535033200
51,243848,243828.0,68.77861,195.23487,(POLYGON ((327055.3644000003 4843861.246400001...,24,25,14,5350332.00,0332.00,35,Ontario,535,35535,Toronto,B,5720.576958,1.525748e+06,535033200.0,535033200
140,243937,243916.0,91.43131,340.32535,"(POLYGON ((327256.0603999998 4843881.4572, 327...",24,25,14,5350332.00,0332.00,35,Ontario,535,35535,Toronto,B,5720.576958,1.525748e+06,535033200.0,535033200
149,243946,243925.0,51.67183,166.87259,(POLYGON ((327535.9248000002 4843873.953600001...,24,25,14,5350332.00,0332.00,35,Ontario,535,35535,Toronto,B,5720.576958,1.525748e+06,535033200.0,535033200
171,243967,243948.0,104.32314,407.19957,(POLYGON ((327616.7039999999 4843875.279999999...,24,25,14,5350332.00,0332.00,35,Ontario,535,35535,Toronto,B,5720.576958,1.525748e+06,535033200.0,535033200
183,243979,243961.0,33.93166,71.88676,(POLYGON ((326852.3279999997 4843875.103599999...,24,25,14,5350332.00,0332.00,35,Ontario,535,35535,Toronto,B,5720.576958,1.525748e+06,535033200.0,535033200
268,244064,244046.0,74.97299,271.19692,(POLYGON ((327246.4939999999 4843873.409600001...,24,25,14,5350332.00,0332.00,35,Ontario,535,35535,Toronto,B,5720.576958,1.525748e+06,535033200.0,535033200
314,244111,244092.0,63.43625,217.34445,(POLYGON ((327163.8949999996 4843874.448000001...,24,25,14,5350332.00,0332.00,35,Ontario,535,35535,Toronto,B,5720.576958,1.525748e+06,535033200.0,535033200
339,244136,244117.0,60.52392,195.84446,(POLYGON ((327129.9044000003 4843890.046800001...,24,25,14,5350332.00,0332.00,35,Ontario,535,35535,Toronto,B,5720.576958,1.525748e+06,535033200.0,535033200


In [17]:
census_buildings_cleaned['geom'] = census_buildings_cleaned['wkb_geometry'].apply(lambda x: WKTElement(x.wkt, srid=2019))

#drop the geometry column as it is now duplicative
census_buildings_cleaned.drop('wkb_geometry', 1, inplace=True)


In [18]:
census_buildings_cleaned.dtypes

ogc_fid               int64
objectid            float64
shape_leng_left     float64
shape_area_left     float64
index_right           int64
gid                   int64
__gid                 int64
ctuid                object
ctname               object
pruid                object
prname               object
cmauid               object
cmapuid              object
cmaname              object
cmatype              object
shape_leng_right    float64
shape_area_right    float64
ctnum               float64
geotext              object
geom                 object
dtype: object

In [19]:
census_buildings_cleaned.to_sql('census_buildings_cleaned', walkability_engine, if_exists='replace', index=True, 
                         dtype={'geom': Geometry('MULTIPOLYGON', srid=2019)})